# Analysing CIA Factbook Data Using SQL

## Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contins demographic information like the following:

* `population` - the global population.
* `population_growth` - the annual population growth rate, as a percentage.
* `area` - the total land and water area.

## Overview of the Data

In [3]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Writing query to return information on the tables in the database:

In [7]:
%%sql
SELECT *
    FROM sqlite_master
    WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


First five rows of the `facts` table in the database:

In [8]:
%%sql
SELECT *
    FROM 'facts'
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


A description for the columns are as follows:

* `name` - the name of the country.
* `area` - the country's total area (both land and water).
* `area_land` - the country's land area in square kilometers.
* `area_water` - the country's water area in square kilometers.
* `population` - the country's population.
* `population_growth` - the country's population growth as a percentage.
* `birth_rate` - the country's birth rate, or the number of births per year per 1,000 people. 
* `death_rate` - the country's death rate, or the number of deaths per year per 1,000 people.

## Summary Statistics

In [12]:
%%sql
SELECT MIN(population) AS 'min_pop', MAX(population) AS 'max_pop', 
       MIN(population_growth) AS 'min_pop_growth', MAX(population_growth) AS 'max_pop_growth'
    FROM facts

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics:

* There's a country with a population of 0
* There's a country with a population of 7256490011 (or more than 7.2 billion people)

## Exploring Outliers

Writing a query that returns the countries with the minimum population:

In [14]:
%%sql
SELECT name, population
    FROM facts
    WHERE population == (SELECT MIN(population)
                         FROM facts
                        );

Done.


name,population
Antarctica,0


Writing a query that returns the countries with the maximum population:

In [15]:
%%sql
SELECT name, population
    FROM facts
    WHERE population == (SELECT MAX(population)
                         FROM facts
                        );

Done.


name,population
World,7256490011


The table contains a row for the whole world, which explains the population of over 7.2 billion. It also contains a row for Antartica, which explains the population of 0. This matches the CIA Factbook [page for Antarctica](https://www.cia.gov/the-world-factbook/countries/antarctica/).

Now that we know this, we will recalculate the summary statistics we calculated earlier - excluding the row for the whole world.

## Summary Statistics Revisited

In [18]:
%%sql
SELECT MIN(population) AS 'min_pop', MAX(population) AS 'max_pop',
       MIN(population_growth) AS 'min_pop_growth', MAX(population_growth) AS 'max_pop_growth'
    FROM facts
    WHERE name != 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


There is a country that has a maximum population of over 1.3 billion. Let's see which country that is:

## Most Populated Country

In [19]:
%%sql
SELECT name, MAX(population) AS 'max_pop'
    FROM facts
    WHERE name <> 'World';

Done.


name,max_pop
China,1367485388


## Country with the Highest Population Growth Rate

In [28]:
%%sql
SELECT name, MAX(population_growth) AS highest_pop_growth
    FROM facts

Done.


name,highest_pop_growth
South Sudan,4.02


The country with the highest population growth rate is South Sudan with 4.02%.

## Top Five Countries with the Lowest Rate of Natural Increase (RNI)

The rate of natural increase (RNI), also known as natural population change, is defined as the birth rate minus the death rate of a particular population, over a particular time period. In this instance it is expressed as a number per 1,000 individuals in the population, per year. It contrasts to total population change by ignoring net migration.

The RNI gives insight into how a region's population is evolving.

In [47]:
%%sql
SELECT name, birth_rate, death_rate, (SELECT birth_rate - death_rate) AS 'RNI'
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY RNI
    LIMIT 5;

Done.


name,birth_rate,death_rate,RNI
Bulgaria,8.92,14.44,-5.52
Serbia,9.08,13.66,-4.58
Latvia,10.0,14.31,-4.3100000000000005
Lithuania,10.1,14.27,-4.17
Ukraine,10.72,14.46,-3.74


Bulgaria has the lowest RNI (-5.52) of all the countries in the CIA World Factbook database.

## Top Five Countries with the Highest Rate of Natural Increase (RNI)

In [48]:
%%sql
SELECT name, birth_rate, death_rate, (SELECT birth_rate - death_rate) AS 'RNI'
    FROM facts
    WHERE birth_rate > death_rate
    ORDER BY RNI DESC
    LIMIT 5;

Done.


name,birth_rate,death_rate,RNI
Malawi,41.56,8.41,33.150000000000006
Uganda,43.79,10.69,33.1
Niger,45.45,12.42,33.03
Burundi,42.01,9.27,32.739999999999995
Mali,44.99,12.89,32.1


Malawi has the highest RNI (33.15).

## Exploring Average Population and Area

Calculating the average value for the `population` and `area` columns (rounded to one decimal place):

In [25]:
%%sql
SELECT ROUND(AVG(population), 1) AS 'avg_pop', ROUND(AVG(area), 1) AS 'avg_area'
    FROM facts
    WHERE name <> 'World';

Done.


avg_pop,avg_area
32242666.6,555093.5


We can see that the average population is around 32 million and the average area is around 555,000 KM.

## Finding Densely Populated Countries

Here we will identify countries that have the following:

* Above-average values for population.
* Below-average values for area.

In [60]:
%%sql
SELECT name, population, area
    FROM facts
    WHERE population > (SELECT AVG(population)
                        FROM facts
                        WHERE name <> 'World')
    AND area < (SELECT AVG(area)
                FROM facts
                WHERE name <> 'World');

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


Many of these countries are well-known to be densely populated (e.g. Japan and Vietnam), so we can be confident with the results.

## Top Five Countries with the Highest Population Density

Population density refers to the number of people living in an area per square kilometre. It is calculated as population divided by total land area.

In [62]:
%%sql
SELECT name, population, area, (SELECT population / area) AS pop_density
    FROM facts
    ORDER BY pop_density DESC
    LIMIT 5;

Done.


name,population,area,pop_density
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191


Macau has the highest population density with 21,168 people per square kilometer.